# **Table of Contents**

0. Setup environment

1. Models

    1.1. DCGAN

    1.2. DCGAN-2D

    1.3. StyleGAN

    1.4. SyleGAN-2D

2. Training

## **0. Setup environment**

In [126]:
!nvidia-smi -L

GPU 0: NVIDIA GeForce RTX 3090 (UUID: GPU-5371946b-bc7a-8404-76f2-94eebc42c1b7)


In [127]:
!tree -d ../image/

../image/
├── Collection1
│   └── BAYC
└── Collection2
    └── EAPES

4 directories


## **1. Models**

### **1.1. DCGAN**

In [128]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

In [129]:
class DCGAN():

    class Generator(nn.Module):
        def __init__(self, channels_noise, channels_img, features_g):
            super().__init__()
            self.gen = nn.Sequential(
                # Input: N x channels_noise x 1 x 1
                self._block(channels_noise, features_g * 16, 4, 1, 0),  # img: 4x4
                self._block(features_g * 16, features_g * 8, 4, 2, 1),  # img: 8x8
                self._block(features_g * 8, features_g * 4, 4, 2, 1),  # img: 16x16
                self._block(features_g * 4, features_g * 2, 4, 2, 1),  # img: 32x32
                nn.ConvTranspose2d(
                    features_g * 2, channels_img, kernel_size=4, stride=2, padding=1
                ),
                # Output: N x channels_img x 64 x 64
                nn.Tanh(),
            )

        def _block(self, in_channels, out_channels, kernel_size, stride, padding):
            return nn.Sequential(
                nn.ConvTranspose2d(
                    in_channels,
                    out_channels,
                    kernel_size,
                    stride,
                    padding,
                    bias=False,
                ),
                nn.BatchNorm2d(out_channels),
                nn.ReLU(),
            )

        def forward(self, x):
            return self.gen(x)

    class Discriminator(nn.Module):
        def __init__(self, channels_img, features_d):
            super().__init__()
            self.disc = nn.Sequential(
                # input: N x channels_img x 64 x 64
                nn.Conv2d(channels_img, features_d, kernel_size=4, stride=2, padding=1),
                nn.LeakyReLU(0.2),
                # _block(in_channels, out_channels, kernel_size, stride, padding)
                self._block(features_d, features_d * 2, 4, 2, 1),
                self._block(features_d * 2, features_d * 4, 4, 2, 1),
                self._block(features_d * 4, features_d * 8, 4, 2, 1),
                # After all _block img output is 4x4 (Conv2d below makes into 1x1)
                nn.Conv2d(features_d * 8, 1, kernel_size=4, stride=2, padding=0),
                nn.Sigmoid(),
            )

        def _block(self, in_channels, out_channels, kernel_size, stride, padding):
            return nn.Sequential(
                nn.Conv2d(
                    in_channels,
                    out_channels,
                    kernel_size,
                    stride,
                    padding,
                    bias=False,
                ),
                nn.BatchNorm2d(out_channels),
                nn.LeakyReLU(0.2),
            )

        def forward(self, x):
            return self.disc(x)

    def __init__(self):
        # Hyperparameters etc.
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.LEARNING_RATE = 2e-4  # could also use two lrs, one for gen and one for disc
        self.BATCH_SIZE = 128
        self.IMAGE_SIZE = 64
        self.CHANNELS_IMG = 3
        self.NOISE_DIM = 100
        self.NUM_EPOCHS = 100
        self.FEATURES_DISC = 64
        self.FEATURES_GEN = 64

        self.transforms = transforms.Compose(
            [
                transforms.Resize((self.IMAGE_SIZE, self.IMAGE_SIZE)),
                transforms.ToTensor(),
                transforms.Normalize(
                    [0.5 for _ in range(self.CHANNELS_IMG)], [0.5 for _ in range(self.CHANNELS_IMG)]
                ),
            ]
        )

        # comment mnist above and uncomment below if train on CelebA
        self.dataset = datasets.ImageFolder(root="../image", transform=self.transforms)
        self.dataloader = DataLoader(self.dataset, batch_size=self.BATCH_SIZE, shuffle=True)
        self.gen = self.Generator(self.NOISE_DIM, self.CHANNELS_IMG, self.FEATURES_GEN).to(self.device)
        self.disc = self.Discriminator(self.CHANNELS_IMG, self.FEATURES_DISC).to(self.device)
        self.initialize_weights(self.gen)
        self.initialize_weights(self.disc)
        
    def initialize_weights(self, model):
        # Initializes weights according to the DCGAN paper
        for m in model.modules():
            if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d, nn.BatchNorm2d)):
                nn.init.normal_(m.weight.data, 0.0, 0.02)

    def test(self):
        N, in_channels, H, W = 8, 3, 64, 64
        noise_dim = 100
        x = torch.randn((N, in_channels, H, W))
        disc = self.Discriminator(in_channels, 8)
        assert disc(x).shape == (N, 1, 1, 1), "Discriminator test failed"
        gen = self.Generator(noise_dim, in_channels, 8)
        self.initialize_weights(gen)
        z = torch.randn((N, noise_dim, 1, 1))
        assert gen(z).shape == (N, in_channels, H, W), "Generator test failed"
        print("Success, tests passed!")

    def train(self):
        opt_gen = optim.Adam(self.gen.parameters(), lr=self.LEARNING_RATE, betas=(0.5, 0.999))
        opt_disc = optim.Adam(self.disc.parameters(), lr=self.LEARNING_RATE, betas=(0.5, 0.999))
        criterion = nn.BCELoss()

        fixed_noise = torch.randn(32, self.NOISE_DIM, 1, 1).to(self.device)
        writer_real = SummaryWriter(f"logsDCGAN/real")
        writer_fake = SummaryWriter(f"logsDCGAN/fake")
        step = 0
        self.gen.train()
        self.disc.train()

        for epoch in range(self.NUM_EPOCHS):
            # Target labels not needed! <3 unsupervised
            for batch_idx, (real, _) in enumerate(self.dataloader):
                real = real.to(self.device)
                noise = torch.randn(self.BATCH_SIZE, self.NOISE_DIM, 1, 1).to(self.device)
                fake = self.gen(noise)

                ### Train Discriminator: max log(D(x)) + log(1 - D(G(z)))
                disc_real = self.disc(real).reshape(-1)
                loss_disc_real = criterion(disc_real, torch.ones_like(disc_real))
                disc_fake = self.disc(fake).reshape(-1)
                loss_disc_fake = criterion(disc_fake, torch.zeros_like(disc_fake))
                loss_disc = (loss_disc_real + loss_disc_fake) / 2
                self.disc.zero_grad()
                loss_disc.backward(retain_graph=True)
                opt_disc.step()

                ### Train Generator: min log(1 - D(G(z))) <-> max log(D(G(z))
                output = self.disc(fake).reshape(-1)
                loss_gen = criterion(output, torch.ones_like(output))
                self.gen.zero_grad()
                loss_gen.backward()
                opt_gen.step()

                # Print losses occasionally and print to tensorboard
                if batch_idx % 100 == 0:
                    print(
                        f"Epoch [{epoch}/{self.NUM_EPOCHS}] Batch {batch_idx}/{len(self.dataloader)} \
                        Loss D: {loss_disc:.4f}, loss G: {loss_gen:.4f}"
                    )

                    with torch.no_grad():
                        fake = self.gen(fixed_noise)
                        # take out (up to) 32 examples
                        img_grid_real = torchvision.utils.make_grid(real[:32], normalize=True)
                        img_grid_fake = torchvision.utils.make_grid(fake[:32], normalize=True)

                        writer_real.add_image("Real", img_grid_real, global_step=step)
                        writer_fake.add_image("Fake", img_grid_fake, global_step=step)

                    step += 1

In [130]:
DCGAN = DCGAN()

### **1.2. DCGAN-2D**

In [131]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

In [132]:
class DCGAN_2D():

    class Generator(nn.Module):
        def __init__(self, channels_noise, channels_img, features_g):
            super().__init__()
            self.gen = nn.Sequential(
                # Input: N x channels_noise x 1 x 1
                self._block(channels_noise, features_g * 16, 4, 1, 0),  # img: 4x4
                self._block(features_g * 16, features_g * 8, 4, 2, 1),  # img: 8x8
                self._block(features_g * 8, features_g * 4, 4, 2, 1),  # img: 16x16
                self._block(features_g * 4, features_g * 2, 4, 2, 1),  # img: 32x32
                nn.ConvTranspose2d(
                    features_g * 2, channels_img, kernel_size=4, stride=2, padding=1
                ),
                # Output: N x channels_img x 64 x 64
                nn.Tanh(),
            )

        def _block(self, in_channels, out_channels, kernel_size, stride, padding):
            return nn.Sequential(
                nn.ConvTranspose2d(
                    in_channels,
                    out_channels,
                    kernel_size,
                    stride,
                    padding,
                    bias=False,
                ),
                nn.BatchNorm2d(out_channels),
                nn.ReLU(),
            )

        def forward(self, x):
            return self.gen(x)

    class Discriminator(nn.Module):
        def __init__(self, channels_img, features_d):
            super().__init__()
            self.disc = nn.Sequential(
                # input: N x channels_img x 64 x 64
                nn.Conv2d(channels_img, features_d, kernel_size=4, stride=2, padding=1),
                nn.LeakyReLU(0.2),
                # _block(in_channels, out_channels, kernel_size, stride, padding)
                self._block(features_d, features_d * 2, 4, 2, 1),
                self._block(features_d * 2, features_d * 4, 4, 2, 1),
                self._block(features_d * 4, features_d * 8, 4, 2, 1),
                # After all _block img output is 4x4 (Conv2d below makes into 1x1)
                nn.Conv2d(features_d * 8, 1, kernel_size=4, stride=2, padding=0),
                nn.Sigmoid(),
            )

        def _block(self, in_channels, out_channels, kernel_size, stride, padding):
            return nn.Sequential(
                nn.Conv2d(
                    in_channels,
                    out_channels,
                    kernel_size,
                    stride,
                    padding,
                    bias=False,
                ),
                nn.BatchNorm2d(out_channels),
                nn.LeakyReLU(0.2),
            )

        def forward(self, x):
            return self.disc(x)

    def initialize_weights(self, model):
        # Initializes weights according to the DCGAN paper
        for m in model.modules():
            if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d, nn.BatchNorm2d)):
                nn.init.normal_(m.weight.data, 0.0, 0.02)

    def test(self):
        N, in_channels, H, W = 8, 3, 64, 64
        noise_dim = 100
        x = torch.randn((N, in_channels, H, W))
        disc = self.Discriminator(in_channels, 8)
        assert disc(x).shape == (N, 1, 1, 1), "Discriminator test failed"
        gen = self.Generator(noise_dim, in_channels, 8)
        self.initialize_weights(gen)
        z = torch.randn((N, noise_dim, 1, 1))
        assert gen(z).shape == (N, in_channels, H, W), "Generator test failed"
        print("Success, tests passed!")

    def __init__(self):
        # Hyperparameters etc.
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.LEARNING_RATE = 2e-4  # could also use two lrs, one for gen and one for disc
        self.BATCH_SIZE = 128
        self.IMAGE_SIZE = 64
        self.CHANNELS_IMG = 3
        self.NOISE_DIM = 100
        self.NUM_EPOCHS = 5
        self.FEATURES_DISC = 64
        self.FEATURES_GEN = 64

        self.transforms = transforms.Compose(
            [
                transforms.Resize((self.IMAGE_SIZE, self.IMAGE_SIZE)),
                transforms.ToTensor(),
                transforms.Normalize(
                    [0.5 for _ in range(self.CHANNELS_IMG)], [0.5 for _ in range(self.CHANNELS_IMG)]
                ),
            ]
        )

        # comment mnist above and uncomment below if train on CelebA
        self.dataset1 = datasets.ImageFolder(root="../image/Collection1", transform=self.transforms)
        self.dataset2 = datasets.ImageFolder(root="../image/Collection2", transform=self.transforms)
        self.dataloader1 = DataLoader(self.dataset1, batch_size=self.BATCH_SIZE, shuffle=True)
        self.dataloader2 = DataLoader(self.dataset2, batch_size=self.BATCH_SIZE, shuffle=True)
        self.gen = self.Generator(self.NOISE_DIM, self.CHANNELS_IMG, self.FEATURES_GEN).to(self.device)
        self.disc1 = self.Discriminator(self.CHANNELS_IMG, self.FEATURES_DISC).to(self.device)
        self.disc2 = self.Discriminator(self.CHANNELS_IMG, self.FEATURES_DISC).to(self.device)
        self.initialize_weights(self.gen)
        self.initialize_weights(self.disc1)
        self.initialize_weights(self.disc2)

    def train(self):
        opt_gen = optim.Adam(self.gen.parameters(), lr=self.LEARNING_RATE, betas=(0.5, 0.999))
        opt_disc1 = optim.Adam(self.disc1.parameters(), lr=self.LEARNING_RATE, betas=(0.5, 0.999))
        opt_disc2 = optim.Adam(self.disc2.parameters(), lr=self.LEARNING_RATE, betas=(0.5, 0.999))
        criterion = nn.BCELoss()

        fixed_noise = torch.randn(32, self.NOISE_DIM, 1, 1).to(self.device)
        writer_real1 = SummaryWriter(f"logsDCGAN_2D/real1")
        writer_real2 = SummaryWriter(f"logsDCGAN_2D/real2")
        writer_fake = SummaryWriter(f"logsDCGAN_2D/fake")
        step = 0
        self.gen.train()
        self.disc1.train()
        self.disc2.train()

        for epoch in range(self.NUM_EPOCHS):
            # Target labels not needed! <3 unsupervised
            batch_idx = 0
            for (real1, _) in self.dataloader1:
                for (real2, _) in self.dataloader2:
                    real1 = real1.to(self.device)
                    real2 = real2.to(self.device)
                    noise = torch.randn(self.BATCH_SIZE, self.NOISE_DIM, 1, 1).to(self.device)
                    fake = self.gen(noise)

                    ### Train Discriminator: max log(D(x)) + log(1 - D(G(z)))
                    disc_real1 = self.disc1(real1).reshape(-1)
                    disc_real2 = self.disc2(real2).reshape(-1)
                    loss_disc_real1 = criterion(disc_real1, torch.ones_like(disc_real1))
                    loss_disc_real2 = criterion(disc_real2, torch.ones_like(disc_real2))
                    disc_fake1 = self.disc1(fake).reshape(-1)
                    disc_fake2 = self.disc2(fake).reshape(-1)
                    loss_disc_fake1 = criterion(disc_fake1, torch.zeros_like(disc_fake1))
                    loss_disc_fake2 = criterion(disc_fake2, torch.zeros_like(disc_fake2))
                    loss_disc1 = (loss_disc_real1 + loss_disc_fake1) / 2
                    loss_disc2 = (loss_disc_real2 + loss_disc_fake2) / 2
                    self.disc1.zero_grad()
                    self.disc2.zero_grad()
                    loss_disc1.backward(retain_graph=True)
                    loss_disc2.backward(retain_graph=True)
                    opt_disc1.step()
                    opt_disc2.step()

                    ### Train Generator: min log(1 - D(G(z))) <-> max log(D(G(z))
                    output1 = self.disc1(fake).reshape(-1)
                    output2 = self.disc2(fake).reshape(-1)
                    loss_gen = (criterion(output1, torch.ones_like(output1)) + criterion(output2, torch.ones_like(output2))) / 2
                    self.gen.zero_grad()
                    loss_gen.backward()
                    opt_gen.step()

                    # Print losses occasionally and print to tensorboard
                    if batch_idx % 100 == 0:
                        print(
                            f"Epoch [{epoch}/{self.NUM_EPOCHS}] Batch {batch_idx}/{len(self.dataloader1)*len(self.dataloader2)} \
                            Loss D1: {loss_disc1:.4f}, Loss D2: {loss_disc2:.4f}, loss G: {loss_gen:.4f}"
                        )

                        with torch.no_grad():
                            fake = self.gen(fixed_noise)
                            # take out (up to) 32 examples
                            img_grid_real1 = torchvision.utils.make_grid(real1[:32], normalize=True)
                            img_grid_real2 = torchvision.utils.make_grid(real2[:32], normalize=True)
                            img_grid_fake = torchvision.utils.make_grid(fake[:32], normalize=True)

                            writer_real1.add_image("Real", img_grid_real1, global_step=step)
                            writer_real2.add_image("Real", img_grid_real2, global_step=step)
                            writer_fake.add_image("Fake", img_grid_fake, global_step=step)

                        step += 1
                    batch_idx += 1

In [133]:
DCGAN_2D = DCGAN_2D()

### **1.3. SyleGAN**

### **1.4. SyleGAN-2D**

## **2. Training**

In [ ]:
import multiprocessing
process1 = multiprocessing.Process(target=DCGAN.train)
process2 = multiprocessing.Process(target=DCGAN_2D.train)
process1.start()
process2.start()
process1.join()
process2.join()